##Connecting Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import zipfile
audio = zipfile.ZipFile('/content/gdrive/MyDrive/data.zip', 'r')
audio.extractall('/content')

##Meta creation

In [ ]:
import pandas as pd
from pathlib import Path
import os
from tqdm.notebook import tqdm

data_folder = Path('/content/val') # specify train or validation folder
df = []

for path, _, files in tqdm(os.walk(data_folder)):
    for file in files:
      
      folder = path.split('/')[-3]
      type_ = path.split('/')[-2]
      id = path.split('/')[-1]
      df.append([folder, type_, id, file])

df = pd.DataFrame(df, columns = ['folder', 'type', 'id', 'path'])
df.to_csv('meta_val.csv')

0it [00:00, ?it/s]

##Meta Demonstration

In [ ]:
import pandas as pd

demonstration = pd.read_csv('/content/meta_train.csv')
demonstration.head()

,Unnamed: 0,folder,type,id,path
0,0,train,noisy,6625,6625_70995_6625-70995-0008.npy
1,1,train,noisy,6625,6625_70995_6625-70995-0004.npy
2,2,train,noisy,6625,6625_39674_6625-39674-0024.npy
3,3,train,noisy,6625,6625_39680_6625-39680-0034.npy
4,4,train,noisy,6625,6625_70995_6625-70995-0006.npy


##Sound Dataset

In [ ]:
from torch.utils.data import Dataset
import numpy as np
import os
from tqdm.notebook import tqdm

class SoundDataset(Dataset):

    def __init__(self, meta, source_folder, transforms):

        self.source_folder = source_folder
        self.transforms = transforms
        self.meta = meta
        self.files = self.meta.path.unique()

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):

        sound = self.files[i]
        sound_df = self.meta.loc[self.meta.path == sound]
        
        noisy_df = sound_df.loc[sound_df.type == 'noisy'].iloc[0]
        clean_df = sound_df.loc[sound_df.type == 'clean'].iloc[0]

        noisy_path = os.path.join(self.source_folder, noisy_df.get('folder'), noisy_df.get('type'), str(noisy_df.get('id')), sound)
        clean_path = os.path.join(self.source_folder, clean_df.get('folder'), clean_df.get('type'), str(clean_df.get('id')), sound)
        
        noisy = np.expand_dims(np.load(noisy_path).astype(float), 2)
        clean = np.expand_dims(np.load(clean_path).astype(float), 2)

        augmented = self.transforms(image=noisy, mask=clean)

        return augmented['image'][0].unsqueeze(0).float(), augmented['mask'][:, :, 0].unsqueeze(0).float()
        

##Separation of meta: Training (80%), validation (20%)

In [ ]:
import pandas as pd

df = pd.read_csv('/content/meta_train.csv', index_col=0) # Specify mata train

df = df.sort_values(by=['id'], ignore_index=True)

train_size = int(0.8 * df.shape[0])
val_size = df.shape[0] - train_size

border_id = df['id'][train_size]

train_df = df[df['id'] < border_id]
val_df = df[df['id'] >= border_id]

##Train and Validation step

In [ ]:
def train_step(images, labels, architecture, criterion, optimizer):
    device = ('cuda' if torch.cuda.is_available() else 'cpu')
    images, labels = images.to(device), labels.to(device)
    architecture.train() # enforce training regime

    pred_logits = architecture(images)
    loss = criterion(pred_logits, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.data.cpu().numpy()

def val_step(images, labels, architecture, criterion, optimizer):
    device = ('cuda' if torch.cuda.is_available() else 'cpu')
    images, labels = images.to(device), labels.to(device)
    architecture.eval() # enforce evaluation regime

    pred_logits = architecture(images)
    loss = criterion(pred_logits, labels)


    return loss.data.cpu().numpy()

##Initialize Dataset, Dataloader

In [ ]:
from torch.utils.data import DataLoader, Dataset
import albumentations as albu
from albumentations.pytorch import ToTensorV2

train_transform = albu.Compose([albu.PadIfNeeded(480, 80),
            albu.RandomCrop(480, 80),
            albu.Resize(576, 96),
            ToTensorV2()])

train_dataset = SoundDataset(train_df, '/content/data', transforms = train_transform)
train_loader = torch.utils.data.DataLoader(train_dataset,
                                             batch_size=32,
                                             num_workers=1, shuffle = True)
val_dataset = SoundDataset(val_df, '/content/data', transforms = train_transform)
val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=32,
                                             num_workers=1, shuffle = False)

In [ ]:
!pip install segmentation-models-pytorch

##Train

In [ ]:
import segmentation_models_pytorch as smp
import torch
import torch.nn as nn
import torch.optim as optim

device = ('cuda' if torch.cuda.is_available() else 'cpu')

model = smp.Unet(encoder_name='resnet18', in_channels=1).to(device)

criterion = nn.MSELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.0001)

for epoch in range(50):
    epoch_loss_train = 0
    epoch_loss_val = 0
    for X_batch, y_batch in tqdm(train_loader):

        loss_train = train_step(X_batch, y_batch, model, criterion, optimizer)
        
        epoch_loss_train += loss_train     

    for X_batch, y_batch in tqdm(val_loader):

        loss_val = val_step(X_batch, y_batch, model, criterion, optimizer)
        
        epoch_loss_val += loss_val 


    print(f'Epoch {epoch+0:03}: | Train_Loss: {epoch_loss_train/len(train_loader):.5f}')
    print(f'Epoch {epoch+0:03}: | Val_Loss: {epoch_loss_val/len(val_loader):.5f}')

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

Epoch 000: | Train_Loss: 0.14807
Epoch 000: | Val_Loss: 0.06397


  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

...

  0%|          | 0/76 [00:00<?, ?it/s]

Epoch 048: | Train_Loss: 0.01584
Epoch 048: | Val_Loss: 0.03132


  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

Epoch 049: | Train_Loss: 0.01535
Epoch 049: | Val_Loss: 0.03106


##Save weights

In [ ]:
import torch

torch.save(model.state_dict(), '/content/gdrive/MyDrive/unet_12.pth')

##Test

In [ ]:
!pip install segmentation-models-pytorch
!pip install torchmetrics

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import zipfile
audio = zipfile.ZipFile('/content/gdrive/MyDrive/data.zip', 'r')
audio.extractall('/content')

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import segmentation_models_pytorch as smp
import albumentations as albu
from albumentations.pytorch import ToTensorV2
import pandas as pd
from pathlib import Path
import os
from tqdm.notebook import tqdm
import numpy as np
from torchmetrics import MeanSquaredError

In [6]:
class SoundDataset(Dataset):

    def __init__(self, meta, source_folder, transforms):

        self.source_folder = source_folder
        self.transforms = transforms
        self.meta = meta
        self.files = self.meta.path.unique()

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):

        sound = self.files[i]
        sound_df = self.meta.loc[self.meta.path == sound]
        
        noisy_df = sound_df.loc[sound_df.type == 'noisy'].iloc[0]
        clean_df = sound_df.loc[sound_df.type == 'clean'].iloc[0]

        noisy_path = os.path.join(self.source_folder, noisy_df.get('folder'), noisy_df.get('type'), str(noisy_df.get('id')), sound)
        clean_path = os.path.join(self.source_folder, clean_df.get('folder'), clean_df.get('type'), str(clean_df.get('id')), sound)
        
        noisy = np.expand_dims(np.load(noisy_path).astype(float), 2)
        clean = np.expand_dims(np.load(clean_path).astype(float), 2)

        augmented = self.transforms(image=noisy, mask=clean)

        return augmented['image'][0].unsqueeze(0).float(), augmented['mask'][:, :, 0].unsqueeze(0).float()
        

In [8]:
from torchmetrics import MeanSquaredError

test_df = pd.read_csv('/content/meta_val.csv') # Specify test meta
test_transform = albu.Compose([albu.PadIfNeeded(480, 80),
            albu.RandomCrop(480, 80),
            albu.Resize(576, 96),
            ToTensorV2()])

test_dataset = SoundDataset(test_df, '/content/data', transforms = test_transform)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                             batch_size=1,
                                             num_workers=1)

device = ("cuda" if torch.cuda.is_available() else 'cpu')

model = smp.Unet(encoder_name='resnet18', in_channels=1).to(device)
model.load_state_dict(torch.load('/content/gdrive/MyDrive/unet_12.pth')) # Specify weights
model.eval()

metric = MeanSquaredError().to(device)

MSE = list()

with torch.no_grad():
  for x, y in tqdm(test_loader):

      x, y = x.to(device), y.to(device)
      y_pred = model(x)
      MSE.append(metric(y_pred, y).to('cpu').detach().numpy())

print('MSE = ', np.mean(MSE))

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

MSE =  0.027801605
